<a href="https://colab.research.google.com/github/owaisorakzai/NLP/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import math,copy
import warnings
from torch import Tensor, nn
import torch.optim as optim
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [ ]:

!pip install -U torch==1.8.0 torchtext==0.9.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 735.5 MB 13 kB/s 
     |████████████████████████████████| 7.1 MB 31.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.


In [ ]:
urdu_txt = open("Urdu.txt", encoding="utf8").read().split("\n")
eng_txt = open("English.txt", encoding="utf8").read().split("\n")
raw_data = {
    "Urdu": [line for line in urdu_txt[1:100]],
    "English": [line for line in eng_txt[1:100]],
}

df = pd.DataFrame(raw_data, columns=["English", "Urdu"])

# create train and test set
train, test = train_test_split(df, test_size=0.1)
train.to_json("train.json", orient="records", lines=True)
test.to_json("test.json", orient="records", lines=True)

train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)
spacy_eng = spacy.load("en")
spacy_ur = spacy.blank('ur')

In [ ]:
import pandas as pd
import torch.nn as nn
import random
import re
import spacy
from torchtext.legacy import data
from spacy.tokenizer import Tokenizer
from spacy.lang.ur import Urdu

import numpy as np

random.seed(0)
df = pd.read_csv("train.csv",names=["English","Urdu"])
df['English'].replace('', np.nan, inplace=True)
df['Urdu'].replace('', np.nan, inplace=True)
df.dropna(subset=['English'], inplace=True)
df.dropna(subset=['Urdu'], inplace=True)
print(df)
'''
First :
python -m spacy download en_core_web_sm
'''
spacy_eng = spacy.load("en_core_web_sm")

ar = Urdu()
ar_Tokenizer = Tokenizer(ar.vocab)

def engTokenizer(text):
 return  [word.text for word in spacy_eng.tokenizer(text)] 

def arTokenizer(sentence):
    return  [word.text for word in 
             ar_Tokenizer(re.sub(r"\s+"," ",re.sub(r"[\.\'\"\n+]"," ",sentence)).strip())]



SRC = data.Field(tokenize=arTokenizer,batch_first=False,tokenizer_language="ar",init_token="ببدأ",eos_token="نهها")
TRG = data.Field(tokenize=engTokenizer,batch_first=False,init_token="<sos>",eos_token="<eos>")
 

class TextDataset(data.Dataset):

    def __init__(self, df, src_field, target_field, is_test=False, **kwargs):
        fields = [('eng',target_field ), ('ar',src_field)]
        samples = []
        for i, row in df.iterrows():
            eng = row.English 
            ar = row.Urdu
            samples.append(data.Example.fromlist([eng, ar], fields))

        super().__init__(samples, fields, **kwargs)
    
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

torchdataset = TextDataset(df,SRC,TRG)

train_data, valid_data = torchdataset.split(split_ratio=0.8, random_state = random.seed(0))

SRC.build_vocab(train_data,min_freq=2)
TRG.build_vocab(train_data,min_freq=2)


                                              English  \
0                                             English   
1             To show anger after getting embarrassed   
2                           A wolf in lamb's clothing   
3               Cut your coat according to your cloth   
4   Success is not final, failure is not fatal: it...   
..                                                ...   
85                               Old mare, red reigns   
86                  Pure gold does not fear the flame   
87  The enemy of a friend is an enemy; the enemy o...   
88               To go about the same old beaten path   
89                         Life is not a bed of roses   

                                                 Urdu  
0                                                Urdu  
1                              کھسیانی بلی کھمبا نوچے  
2                         بخل میں چھری منہ پے رام رام  
3                        جتنی چادر ہو اتنا پیر پھیلاو  
4   نہ کامیابی حتمی ہوتی ہے اور نہ 

In [ ]:
class Embedder(torch.nn.Module):
    
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, d_model)
        
    def forward(self, x):
        return self.embed(x)

import math

class PositionalEncoder(torch.nn.Module):
    
    def __init__(self, d_model, max_seq_len=80):
        super().__init__()
        self.d_model = d_model
        
        pe_matrix = torch.zeros(max_seq_len, d_model)
        
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe_matrix[pos, i] = math.sin(pos/10000**(2*i/d_model))
                pe_matrix[pos, i+1] = math.cos(pos/10000**(2*i/d_model))
        pe_matrix = pe_matrix.unsqueeze(0) 
        self.register_buffer('pe', pe_matrix)
        
    def forward(self, x):
        seq_len = x.size()[1]
        x = x + self.pe[:, :seq_len]
        return x


import math
import torch.nn.functional as F

def scaled_dot_product_attention(q, k, v, mask=None, dropout=None):
    attention_scores = torch.matmul(q, k.transpose(-2, -1))/math.sqrt(q.shape[-1])
    

    if mask is not None:
        attention_scores = attention_scores.masked_fill(mask == 0, value=-1e9)
        
    attention_weights = F.softmax(attention_scores, dim=-1)
    
    if dropout is not None:
        attention_weights = dropout(attention_weights)
        
    output = torch.matmul(attention_weights, v)
    return output

class MultiHeadAttention(torch.nn.Module):
    def __init__(self, n_heads, d_model, dropout=0.1):
        super().__init__()
        
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = self.d_v = d_model//n_heads
        self.q_linear_layers = []
        self.k_linear_layers = []
        self.v_linear_layers = []
        for i in range(n_heads):
            self.q_linear_layers.append(torch.nn.Linear(d_model, self.d_k))
            self.k_linear_layers.append(torch.nn.Linear(d_model, self.d_k))
            self.v_linear_layers.append(torch.nn.Linear(d_model, self.d_v))
        
        self.dropout = torch.nn.Dropout(dropout)
        self.out = torch.nn.Linear(n_heads*self.d_v, d_model)
        
    def forward(self, q, k, v, mask=None):
        multi_head_attention_outputs = []
        for q_linear, k_linear, v_linear in zip(self.q_linear_layers,
                                                self.k_linear_layers,
                                                self.v_linear_layers):
            new_q = q_linear(q)
            new_k = k_linear(k)
            new_v = v_linear(v)
            head_v = scaled_dot_product_attention(new_q, new_k, new_v, mask, self.dropout)
            multi_head_attention_outputs.append(head_v)
        concat = torch.cat(multi_head_attention_outputs, -1)
        output = self.out(concat)
        
        return output



class FeedForward(torch.nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout=0.1):
        super().__init__()
        
        self.linear_1 = torch.nn.Linear(d_model, d_ff)
        self.dropout = torch.nn.Dropout(dropout)
        self.linear_2 = torch.nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

class LayerNorm(torch.nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.d_model = d_model
        self.alpha = torch.nn.Parameter(torch.ones(self.d_model))
        self.beta = torch.nn.Parameter(torch.zeros(self.d_model))
        self.eps = eps
        
    def forward(self, x):
        x_hat = (x - x.mean(dim=-1, keepdim=True))/(x.std(dim=-1, keepdim=True) + self.eps)
        x_tilde = self.alpha*x_hat + self.beta
        return x_tilde
class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.norm_1 = LayerNorm(d_model)
        self.norm_2 = LayerNorm(d_model)
        self.multi_head_attention = MultiHeadAttention(n_heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout_1 = torch.nn.Dropout(dropout)
        self.dropout_2 = torch.nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x = x + self.dropout_1(self.multi_head_attention(x, x, x, mask))
        x = self.norm_1(x)
        
        x = x + self.dropout_2(self.feed_forward(x))
        x = self.norm_2(x)
        return x



class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.norm_1 = LayerNorm(d_model)
        self.norm_2 = LayerNorm(d_model)
        self.norm_3 = LayerNorm(d_model)
        
        self.dropout_1 = torch.nn.Dropout(dropout)
        self.dropout_2 = torch.nn.Dropout(dropout)
        self.dropout_3 = torch.nn.Dropout(dropout)
        
        self.multi_head_attention_1 = MultiHeadAttention(n_heads, d_model)
        self.multi_head_attention_2 = MultiHeadAttention(n_heads, d_model)
        
        self.feed_forward = FeedForward(d_model)
        
    def forward(self, x, encoder_output, src_mask, trg_mask):
        x = self.dropout_1(self.multi_head_attention_1(x, x, x, trg_mask))
        x = x + self.norm_1(x)
        
        x = self.dropout_2(self.multi_head_attention_2(x, encoder_output, encoder_output, src_mask))
        x = x + self.norm_2(x)
        
        x = self.dropout_3(self.feed_forward(x))
        x = x + self.norm_3(x)
        
        return x

import copy

def clone_layer(module, N):
    return torch.nn.ModuleList([copy.deepcopy(module) for i in range(N)])
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads):
        super().__init__()
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.encoder_layers = clone_layer(EncoderLayer(d_model, n_heads), N)
        self.norm = LayerNorm(d_model)
        
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for encoder in self.encoder_layers:
            x = encoder(x, mask)
        return self.norm(x)


class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads):
        super().__init__()
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.decoder_layers = clone_layer(DecoderLayer(d_model, n_heads), N)
        self.norm = LayerNorm(d_model)
        
    def forward(self, trg, encoder_output, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for decoder in self.decoder_layers:
            x = decoder(x, encoder_output, src_mask, trg_mask)
        return self.norm(x)



class Transformer(torch.nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, N, n_heads):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, N, n_heads)
        self.decoder = Decoder(trg_vocab_size, d_model, N, n_heads)
        self.linear = torch.nn.Linear(d_model, trg_vocab_size)
        
    def forward(self, src, trg, src_mask, trg_mask):
        encoder_output = self.encoder(src, src_mask)
        decoder_output = self.decoder(trg, encoder_output, src_mask, trg_mask)
        output = self.linear(decoder_output)
        return output





In [ ]:
d_model = 512
n_heads = 8
N = 6
src_vocab_size = len(SRC.vocab)
trg_vocab_size = len(TRG.vocab)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Size of urdu vocabulary:",src_vocab_size)
train_iter, valid_iter = data.BucketIterator.splits(
    (train_data,valid_data), 
    batch_size = 16,
    sort=None,
    sort_within_batch=False,
    sort_key=lambda x: len(x.eng),
    device = device,
    shuffle=True
)
trg_vocab_size =len(TRG.vocab)
print("Size of english vocabulary:",trg_vocab_size)
pad_idx = SRC.vocab.stoi["<pad>"]
model = Transformer( trg_vocab_size,src_vocab_size, d_model, N, n_heads)
for p in model.parameters():
    if p.dim() > 1:
        torch.nn.init.xavier_uniform(p)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
import numpy as np

def create_mask(src_input, trg_input):
    pad = SRC.vocab.stoi['<pad>']
    src_mask = (src_input != pad).unsqueeze(1)
    
    # Target input mask
    trg_mask = (trg_input != pad).unsqueeze(1)
    
    seq_len = trg_input.size(1)
    nopeak_mask = np.tril(np.ones((1, seq_len, seq_len)), k=0).astype('uint8')
    nopeak_mask = torch.from_numpy(nopeak_mask) != 0
    trg_mask = trg_mask & nopeak_mask
    
    return src_mask, trg_mask

Size of urdu vocabulary: 70
Size of english vocabulary: 78


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [ ]:

import time

def train_model(n_epochs, output_interval=100):
    model.train()
    start = time.time()
    
    for epoch in range(n_epochs):
        
        total_loss = 0
        for i, batch in enumerate(train_iter):
            
            src_input = batch.eng.transpose(0, 1)
            trg = batch.ar.transpose(0, 1) 
            
            trg_input = trg[:, :-1]
            ys = trg[:, 1:].contiguous().view(-1)
            
            src_mask, trg_mask = create_mask(src_input, trg_input)
            preds = model(src_input, trg_input, src_mask, trg_mask)
            
            optimizer.zero_grad()
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=1)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data

            if (i + 1) % output_interval == 0:
                avg_loss = total_loss/output_interval
                print('time = {}, epoch = {}, iter = {}, loss = {}'.format((time.time() - start)/60,
                                                                           epoch + 1,
                                                                           i + 1,
                                                                           avg_loss))
                total_loss = 0
                start = time.time()
train_model(3, output_interval=1)






time = 0.14562635421752929, epoch = 1, iter = 1, loss = 4.582399845123291
time = 0.08033700386683146, epoch = 1, iter = 2, loss = 5.814487934112549
time = 0.053155708312988284, epoch = 1, iter = 3, loss = 3.2108092308044434
time = 0.03178740342458089, epoch = 1, iter = 4, loss = 3.3721091747283936
time = 0.025288299719492594, epoch = 1, iter = 5, loss = 3.4064455032348633
time = 0.02400938669840495, epoch = 2, iter = 1, loss = 2.697145938873291
time = 0.030090534687042238, epoch = 2, iter = 2, loss = 3.408078908920288
time = 0.03930020332336426, epoch = 2, iter = 3, loss = 2.696653127670288
time = 0.06033289432525635, epoch = 2, iter = 4, loss = 2.9975969791412354
time = 0.04249429702758789, epoch = 2, iter = 5, loss = 3.4456493854522705
time = 0.04225571950276693, epoch = 3, iter = 1, loss = 2.8997251987457275
time = 0.031193641821543376, epoch = 3, iter = 2, loss = 3.281088352203369
time = 0.031675843397776286, epoch = 3, iter = 3, loss = 2.7641992568969727
time = 0.05299049218495687